In [ ]:
# convert our shapefile to GeoJSON
!ogr2ogr -f GeoJSON gin_adm/GIN_adm2.json gin_adm/GIN_adm2.shp

In [ ]:
%matplotlib inline

In [ ]:
import json

import numpy as np
import folium
import pandas as pd

In [ ]:
geo_json_data = json.load(open('gin_adm/GIN_adm2.json'))

In [ ]:
m = folium.Map([9.886945, -11.251629], zoom_start=7)
folium.GeoJson(geo_json_data).add_to(m)
m

In [ ]:
# Grab all prefecture names from the file containing
# their shapes
prefectures = folium.GeoJson(geo_json_data)
prefecture_names = [pref['properties']['NAME_2'] for pref in prefectures.data['features']]

In [ ]:
# Reformat our data to have two columns
df = pd.read_csv("guinea-ebola.csv")
# select interesting prefectures
prefectures = list(df.columns[3:])

# take the entry at the last recorded day
# as these are cumulative deaths it is also
# the maximum. Some areas used to be merged
# but then started reporting separately, they
# will be NaN, so we drop them
data = []
for pref in prefecture_names:
    if pref in df:
        deaths = df[pref].iloc[-1]
    else:
        deaths = 1.
    data.append((pref, deaths))

deaths = pd.DataFrame(data,
                      columns=["Prefectures", "Deaths"])
deaths.dropna(inplace=True)
deaths_dict = deaths.set_index("Prefectures")['Deaths']

In [ ]:
deaths_dict

In [ ]:
step = folium.colormap.linear.YlOrRd.scale(1,505)
step = step.to_step(n=10, data=set(sorted(deaths_dict.values)), method='quant')

m = folium.Map([9.886945, -11.251629], zoom_start=7)
m.add_children(step)
folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': step(deaths_dict[feature['properties']['NAME_2']]),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        },
    ).add_to(m)

m